<h1 style="color:white;background-color:rgb(255, 108, 0);padding-top:1em;padding-bottom:0.7em;padding-left:1em;">Excersise 3.4 solution</h1>
<hr>

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
#Test data
test_data = np.arange(0,1,0.01)

#Load data and plot
inputs,labels = np.load('SIT_visit/Block_3/data/data.npy')

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(inputs, labels, color='black')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Training data')
plt.show()

In [ ]:
#Data pipeline:
batch_size = 100

tf.reset_default_graph()

ds = tf.data.Dataset.from_tensor_slices((inputs,labels)).shuffle(len(labels)).repeat().batch(batch_size)

iterator = ds.make_initializable_iterator()
iterator_init = iterator.initializer

inps,labs = iterator.get_next()
inps = tf.reshape(inps, [-1,1], name='inputs')
labs = tf.reshape(labs, [-1,1], name='labels')


#Neural network model:
def net(x, layer_sizes=[10,10,1], reuse=False, name='FC'):
    with tf.variable_scope(name, reuse=reuse):
        hidden_1 = tf.layers.dense(x, layer_sizes[0], activation=tf.nn.tanh, name='hidden_layer_1')
        hidden_2 = tf.layers.dense(hidden_1, layer_sizes[1], activation=tf.nn.tanh, name='hidden_layer_2')
        output = tf.layers.dense(hidden_2, layer_sizes[2], name='output_layer')
        return output

predictions = net(inps)
predictions = tf.identity(predictions, name='predictions')

extra_pred = net(inps, layer_sizes=[20,20,6], name='Extra')
extra_pred = tf.identity(extra_pred, name='extra_pred')
par = tf.reduce_mean(extra_pred, axis=0)

#Loss function and optimization:
loss = tf.identity(tf.losses.mean_squared_error(labels=labs, predictions=predictions), name='loss')

extra_func = par[0]*tf.sin(par[1]*inps+par[2])**2+par[3]*tf.sin(par[4]*inps+par[5])
extra_loss = tf.identity(tf.losses.mean_squared_error(labels=labs, predictions=extra_func), name='extra_loss')

train_step = tf.train.AdamOptimizer(0.1).minimize(loss)
extra_train_step = tf.train.AdamOptimizer(0.1).minimize(extra_loss)

variable_init = tf.global_variables_initializer()


#Training process and evaluation:
train = True
losses = []
extra_losses = []

with tf.Session() as sess:
    sess.run([iterator_init,variable_init])
    c = 0
    while train:
        l,_ = sess.run([loss, train_step])
        print('Batch:', c, 'loss:', l)
        c += 1
        if l < 0.002 or c > 1000:
            train = False
        losses.append(l)
    print('\nEnd of training process\n')
    p = sess.run(predictions, feed_dict={inps: np.reshape(inputs,[-1,1])})
    for j in range(len(inputs)):
        print('\nInputs:', inputs[j], 'prediction:', p[j][0], 'correct label:', labels[j])
    
    print('Extra task:')
    c = 0
    train = True
    while train:
        params,l,_ = sess.run([par,extra_loss, extra_train_step])
        print('Batch:', c, 'loss:', l)
        c += 1
        if l < 0.002 or c > 3000:
            train = False
        extra_losses.append(l)
    print('\nEnd of extra task\nPredicted params:', params)


x = np.arange(0,1,0.01)
y = params[0]*np.sin(params[1]*x+params[2])**2+params[3]*np.sin(params[4]*x+params[5])

#Evaluation
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(inputs, labels, label='Training data', color='black')
ax.scatter(inputs,p, label='Predictions', color='green')
ax.plot(x, y, label='Function with predicted params')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.legend()
plt.title('Evaluation')
plt.show()

#Real params 2,8,3,0.4,5,-1

In [ ]:
#Plot the losses
plt.figure(1)
plt.subplot(211)
plt.plot(range(len(losses)), losses, label='Regression model loss')
plt.ylabel('Loss', fontsize=12)
plt.title('Losses')
#fig_2 = plt.figure(2)
plt.subplot(212)
plt.plot(range(len(extra_losses)), extra_losses, label='')
plt.xlabel('Batch', fontsize=12)
plt.ylabel('Extra task loss', fontsize=12)
plt.show()